# Binary Output in the LSTM

In [5]:
import matplotlib.pyplot as plt
import mlflow
import mlflow.keras
import numpy as np
from numpy import ndarray
import pandas as pd
import sys

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

%load_ext lab_black

sys.path.append("/Users/renero/Documents/SideProjects/trader/src")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/predictor")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/utils")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/indicators")


from cs_dictionary import CSDictionary
from lstm import lstm_1layer, lstm_2layer
from sequences import sequences
from ticks import Ticks
from metrics import metrics

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


#### Read the data, scale and append indicators

In [6]:
argv = [
    "",
    "-c",
    "../src/predictor/params.dax.yaml",
    "-f",
    "../data/^GDAXI/dax_2018_2019.csv",
    "--window",
    "8",
    "--epochs",
    "400",
    "train",
]
params = CSDictionary(args=argv)

ticks = Ticks(params, params.input_file).scale()

ticks.append_indicator(["trend", "median_filter"])
ticks.data.head()

2020-09-18 19:22:12 - INFO  - CSDictionary:__init__           - Using configuration parameters from: ../src/predictor/params.dax.yaml


,open,high,low,close,trend,median_filter
Datetime,,,,,,
2018-01-02,0.71,0.66,0.61,0.67,1.0,0.67
2018-01-03,0.73,0.78,0.77,0.79,1.0,0.79
2018-01-04,0.90,0.99,0.96,1.00,1.0,1.00
2018-01-05,1.07,1.13,1.13,1.17,1.0,1.17
2018-01-08,1.26,1.21,1.26,1.22,1.0,1.17


#### Split the dataset

In [7]:
X_train, y_train, X_test, y_test = ticks.prepare_for_training(
    predict="trend", train_columns=["close", "trend"]
)

#### Train the network and evaluate

In [8]:
nn = lstm_1layer(params, binary=True)
nn.start_training(X_train, y_train, "trend")
nn.evaluate(X_test, y_test)
nn.end_experiment()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               67072     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 67,330
Trainable params: 67,330
Non-trainable params: 0
_________________________________________________________________
2020-09-18 19:22:12 - INFO  - lstm_1layer:_build_model        - NN lstm_1layer created
2020-09-18 19:23:28 - INFO  - lstm_1layer:evaluate            - Predictions (yhat): (48,)
2020-09-18 19:23:28 - INFO  - lstm_1layer:evaluate            - Trend acc.: 0.52
